# `MonotonicSplineEpistasisBottleneckLikelihood` model
Simulates some variants and analyzes them under the `globalepistasis.MonotonicSplineEpistasisBottleneckLikelihood` model.

This notebook is designed as a test, not for user documentation--and it uses some private methods that should not be used in real analyses.

## Set up
Import Python modules:

In [1]:
import cProfile
import math
import pstats
import random
import tempfile
import time
import warnings

import binarymap

import numpy

import pandas as pd

from plotnine import *

import scipy

import dms_variants.globalepistasis
import dms_variants.codonvarianttable
import dms_variants.simulate
from dms_variants.constants import CBPALETTE, CODONS_NOSTOP

Parameters for simulations:

In [2]:
seed = 1  # random number seed
genelength = 30  # gene length in codons
bclen = 16  # length of barcodes
libs = ["lib_1"]  # distinct libraries of gene
variants_per_lib = 1000 * genelength  # variants per library
avgmuts = 2.5  # average codon mutations per variant
variant_error_rate = 0  # rate variants are mis-sequenced
depth = 5000 * variants_per_lib  # per-sample sequencing depth
lib_uniformity = 3  # uniformity of library pre-selection
noise = 0.0  # non-statistical noise in selections
bottleneck = 5 * variants_per_lib
bottlenecks = {  # bottlenecks from pre- to post-selection
    "loose_bottle": bottleneck,
}

Seed random number generators:

In [3]:
random.seed(seed)
numpy.random.seed(seed)

Suppress warnings that clutter output:

In [4]:
warnings.simplefilter("ignore")

## Simulate variants
Simulate wildtype gene sequence:

In [5]:
geneseq = "".join(random.choices(CODONS_NOSTOP, k=genelength))
print(f"Wildtype gene of {genelength} codons:\n{geneseq}")

Wildtype gene of 30 codons:
AGATCCGTGATTCTGCGTGCTTACACCAACTCACGGGTGAAACGTGTAATCTTATGCAACAACGACTTACCTATCCGCAACATCCGGCTG


Simulate a `CodonVariantTable`:

In [6]:
variants = dms_variants.simulate.simulate_CodonVariantTable(
    geneseq=geneseq,
    bclen=bclen,
    library_specs={
        lib: {"avgmuts": avgmuts, "nvariants": variants_per_lib} for lib in libs
    },
    seed=seed,
)

Number of amino-acid mutations per variant in library:

In [7]:
# NBVAL_IGNORE_OUTPUT

p = variants.plotNumMutsHistogram("aa", samples=None, max_muts=8)
p = p + theme(panel_grid_major_x=element_blank())  # no vertical grid lines
_ = p.draw()

## Simulate variant counts
We will simulate using a `SigmoidPhenotypeSimulator`:

In [8]:
phenosimulator = dms_variants.simulate.SigmoidPhenotypeSimulator(
    geneseq,
    seed=seed,
)

Plot relationship between simulated and observed phenotypes:

In [9]:
# NBVAL_IGNORE_OUTPUT

for value in ["enrichment", "phenotype"]:
    p = phenosimulator.plotLatentVsObserved(value)
    _ = p.draw()

Plot latent and observed phenotypes of all single amino-acid mutants:

In [10]:
# NBVAL_IGNORE_OUTPUT

for value in ["latentPhenotype", "observedPhenotype", "observedEnrichment"]:
    p = phenosimulator.plotMutsHistogram(value)
    _ = p.draw()

Now we simulate the counts according to the observed phenotype:

In [11]:
counts = dms_variants.simulate.simulateSampleCounts(
    variants=variants,
    phenotype_func=phenosimulator.observedEnrichment,
    variant_error_rate=variant_error_rate,
    pre_sample={"total_count": depth, "uniformity": lib_uniformity},
    pre_sample_name="pre-selection",
    post_samples={
        name: {"noise": noise, "total_count": depth, "bottleneck": bottle}
        for name, bottle in bottlenecks.items()
    },
    seed=seed,
)

Add the counts to the variant table:

In [12]:
variants.add_sample_counts_df(counts)

Confirm expected selection on mutations:

In [13]:
# NBVAL_IGNORE_OUTPUT

p = variants.plotNumCodonMutsByType("all")
p = p + theme(panel_grid_major_x=element_blank())  # no vertical grid lines
_ = p.draw()

## Get functional scores
Get the functional scores:

In [14]:
func_scores = variants.func_scores("pre-selection")

Classify the functional scores:

In [15]:
func_scores = variants.classifyVariants(func_scores)

Use `plotnine` to plot distribution of functional scores:

In [16]:
# NBVAL_IGNORE_OUTPUT

p = (
    ggplot(func_scores, aes("variant_class", "func_score"))
    + geom_violin(aes(fill="variant_class"))
    + ylab("functional score")
    + xlab("")
    + facet_grid("post_sample ~ library")
    + theme(
        figure_size=(2.75 * len(libs), 2 * len(bottlenecks)),
        axis_text_x=element_text(angle=90),
        panel_grid_major_x=element_blank(),  # no vertical grid lines
    )
    + scale_fill_manual(values=CBPALETTE[1:], guide=False)
)
_ = p.draw()

## Initialize model

Make a binary map for the variants:

In [17]:
binarymap = binarymap.BinaryMap(func_scores)
print(f"Length of binary map is {binarymap.binarylength}")

Length of binary map is 600


Now initialize the model:

In [18]:
model = dms_variants.globalepistasis.MonotonicSplineEpistasisBottleneckLikelihood(
    binarymap, bottleneck=bottleneck
)

Look at some model attributes:

In [19]:
model.nparams

607

In [20]:
model.likelihood_calc_params_dict

{}

In [21]:
assert numpy.allclose(model.loglik, -193491, rtol=0.01, atol=3), model.loglik

In [22]:
model.latent_phenotype_wt()

0.0

## Fit model
Perform initial least squares optimization followed by rescaling latent effects and epistasis function parameters (normally this is actually performed by the `fit` method):

In [23]:
model._fit_latent_leastsquares()
model._prescale_params(
    k=1, g_k_range=(binarymap.func_scores.min(), binarymap.func_scores.max())
)

Look at model parameters again:

In [24]:
for key, val in model.epistasis_func_params_dict.items():
    print(f"{key}: {round(val, 2)}")

c_alpha: -15.59
alpha_1: 3.72
alpha_2: 3.72
alpha_3: 3.72
alpha_4: 3.72
alpha_5: 3.72


Check the gradients:

In [25]:
err = scipy.optimize.check_grad(
    model._loglik_by_allparams, model._dloglik_by_allparams, model._allparams
)
tot = math.sqrt(sum(model._dloglik_by_allparams(model._allparams) ** 2))
if err / tot > 1e-2:
    raise ValueError(
        f"Numerical and analytical gradients differ: {err}, {tot}, {err / tot}"
    )

Print model parameters after this "partial" fitting:

In [26]:
assert numpy.allclose(model.loglik, -120885, rtol=0.01, atol=3), model.loglik

In [27]:
for param, val in model.likelihood_calc_params_dict.items():
    print(f"{param}: {val:.3f}")

In [28]:
round(model.latent_phenotype_wt(), 1)

0.9

Now fit the model by maximum likelihood, also profiling [as here](https://stackoverflow.com/a/1584468).


In [29]:
# NBVAL_IGNORE_OUTPUT
start = time.time()
profiler = cProfile.Profile()
optres = profiler.runcall(model.fit)
print(f"Fitting took {time.time() - start:.1f} seconds")
print(
    f"Iterations: {optres.nit}.\n"
    f"Function evaluations: {optres.nfev}\n"
    f"Termination message: {optres.message}"
)

Fitting took 46.6 seconds
Iterations: 589.
Function evaluations: 705
Termination message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


Look at some of the model parameters after optimization:

In [30]:
assert numpy.allclose(model.loglik, -63335, rtol=0.01, atol=3), model.loglik

In [31]:
assert numpy.allclose(model.latent_phenotype_wt(), 0), model.latent_phenotype_wt()

Plot correlation of measured functional scores with latent and observed phenotypes:

In [32]:
# NBVAL_IGNORE_OUTPUT

df = pd.DataFrame(
    {
        "measured": binarymap.func_scores,
        "latent": model.phenotypes_frombinary(binarymap.binary_variants, "latent"),
        "observed": model.phenotypes_frombinary(binarymap.binary_variants, "observed"),
    }
)

for phenotype in ["latent", "observed"]:
    _ = (ggplot(df, aes(phenotype, "measured")) + geom_bin2d()).draw()

    if phenotype == "observed":
        r, p = scipy.stats.pearsonr(
            binarymap.func_scores,
            model.phenotypes_frombinary(binarymap.binary_variants, phenotype),
        )
        print(f"\nCorrelation for {phenotype}:\nR = {r:.1f}, P = {p:.2g}")


Correlation for observed:
R = 0.9, P = 0


Plot latent versus observed phenotypes:

In [33]:
# NBVAL_IGNORE_OUTPUT

_ = (ggplot(df, aes("latent", "observed")) + geom_bin2d()).draw()

Look at profiling results:

In [34]:
# NBVAL_IGNORE_OUTPUT
with tempfile.NamedTemporaryFile("w") as f:
    profiler.dump_stats(f.name)
    f.flush()
    stats = pstats.Stats(f.name).strip_dirs()
for stat in ["cumtime", "tottime"]:
    print(stats.sort_stats(stat).print_stats(20))

Tue Feb  6 13:53:52 2024    /tmp/tmp7vbri1ir

         3227834 function calls (2977364 primitive calls) in 46.550 seconds

   Ordered by: cumulative time
   List reduced from 355 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001   46.550   46.550 globalepistasis.py:1446(fit)
        1    0.000    0.000   46.329   46.329 _minimize.py:51(minimize)
        1    0.190    0.190   46.326   46.326 _lbfgsb_py.py:212(_minimize_lbfgsb)
      705    0.013    0.000   46.047    0.065 _differentiable_functions.py:293(fun_and_grad)
     2119    1.698    0.001   31.580    0.015 ispline.py:251(_calculate_Itotal_or_dItotal)
    13215    4.041    0.000   28.173    0.002 ispline.py:706(_calculate_I_or_dI)
      706    0.003    0.000   23.920    0.034 _differentiable_functions.py:260(_update_fun)
      705    0.004    0.000   23.917    0.034 _differentiable_functions.py:162(update_fun)
      705    0.016    0.000   23.913    0

## Examine single mutants
Correlation of predicted and actual predicted latent effects.
Get a data frame of the latent effects of all single mutations:

In [35]:
singles_df = model.latent_effects_df

Add the true latent effects from the simulator:

In [36]:
singles_df = singles_df.rename(columns={"mutation": "aa_substitutions"}).assign(
    true_latent_effect=lambda x: x["aa_substitutions"].map(phenosimulator.muteffects)
)

Now add the predicted and true latent and observed phenotypes:

In [37]:
singles_df = model.add_phenotypes_to_df(singles_df)

singles_df = singles_df.assign(
    true_latent_phenotype=lambda x: x["aa_substitutions"].map(
        phenosimulator.latentPhenotype
    ),
    true_enrichment=lambda x: x["aa_substitutions"].map(
        phenosimulator.observedEnrichment
    ),
    true_observed_phenotype=lambda x: x["aa_substitutions"].map(
        phenosimulator.observedPhenotype
    ),
    enrichment=lambda x: model.enrichments(x["observed_phenotype"]),
)

Correlations among the observed quantities:

In [38]:
print(
    singles_df[[c for c in singles_df.columns if "latent" not in c]]
    .corr(numeric_only=True)
    .to_csv(float_format="%.1f")
)

,observed_phenotype,true_enrichment,true_observed_phenotype,enrichment
observed_phenotype,1.0,0.9,1.0,0.9
true_enrichment,0.9,1.0,0.9,1.0
true_observed_phenotype,1.0,0.9,1.0,0.9
enrichment,0.9,1.0,0.9,1.0



## Examine all variants used for fitting
Look at correlation among all the variants used to fit the model:

In [39]:
variants_df = model.phenotypes_df

In [40]:
# NBVAL_IGNORE_OUTPUT

_ = (ggplot(variants_df, aes("func_score", "observed_phenotype")) + geom_bin2d()).draw()

In [41]:
print(
    variants_df[["func_score", "latent_phenotype", "observed_phenotype"]]
    .corr()
    .to_csv(float_format="%.1f")
)

,func_score,latent_phenotype,observed_phenotype
func_score,1.0,0.7,0.9
latent_phenotype,0.7,1.0,0.8
observed_phenotype,0.9,0.8,1.0

